## Interaktiv analyse med dash app

In [1]:
import sys
sys.path.append("../analysis")
from analyses import DataAnalyzer

import plotly.graph_objects as go
from dash import Dash, html, dcc, Input, Output
import calendar

analyzer = DataAnalyzer("../../data/processed")

cities = ["oslo", "tromso"]
element_ids = [
    "sum(precipitation_amount P1D)",
    "mean(air_temperature P1D)",
    "mean(wind_speed P1D)",
    "max(air_temperature P1D)",
    "min(air_temperature P1D)",
    "range(air_temperature P1D)"
]

app = Dash(__name__)
app.title = "Klimatologi med/uten outliers"

app.layout = html.Div(
    style={"maxWidth": "900px", "margin": "auto"},
    children=[
        html.H1("Klimatologisk Månedsmiddel"),

        html.Label("Måletype (elementId):"),
        dcc.Dropdown(
            id="element-dropdown",
            options=[{"label": e, "value": e} for e in element_ids],
            value=element_ids[0],
            clearable=False,
        ),

        html.Label("Statistikk:"),
        dcc.RadioItems(
            id="stat-radio",
            options=[
                {"label": "Gjennomsnitt", "value": "mean"},
                {"label": "Median", "value": "median"},
                {"label": "Std.avvik", "value": "std"},
            ],
            value="mean",
            inline=True,
        ),

        html.Label("Outliers:"),
        dcc.RadioItems(
            id="outlier-radio",
            options=[
                {"label": "Ta med", "value": "med"},
                {"label": "Uten", "value": "uten"},
            ],
            value="med",
            inline=True,
        ),

        html.Label("By(er):"),
        dcc.Checklist(
            id="city-checklist",
            options=[{"label": c.capitalize(), "value": c} for c in cities],
            value=cities,
            inline=True,
        ),

        dcc.Graph(id="climate-graph"),
    ],
)

@app.callback(
    Output("climate-graph", "figure"),
    Input("element-dropdown", "value"),
    Input("stat-radio", "value"),
    Input("outlier-radio", "value"),
    Input("city-checklist", "value"),
)
def update_graph(element_id, statistikk, outlier_choice, selected_cities):

    fig = go.Figure()
    month_order = [calendar.month_abbr[m].capitalize() for m in range(1, 13)]
    remove = (outlier_choice == "uten")

    for city in selected_cities:
        df_klima = analyzer.langtidsmiddel_per_måned(
            by=city,
            element_id=element_id,
            remove_outliers=remove,
            statistikk=statistikk,
        )
        df_klima = df_klima.sort_values("month")  # 1..12

        fig.add_trace(
            go.Bar(
                x=df_klima["month_name"],
                y=df_klima["verdi"],
                name=city.capitalize(),
            )
        )

    t_out = "uten outliers" if remove else "med outliers"
    fig.update_layout(
        title=f"Klimatologisk {statistikk} per måned – {element_id} ({t_out})",
        xaxis_title="Måned",
        yaxis_title="Verdi",
        barmode="group",
        template="plotly_white",
        xaxis=dict(categoryorder="array", categoryarray=month_order),
    )
    return fig


if __name__ == "__main__":
    app.run(debug=True, port=8051)

### Sjekke data for bestemt dag basert på historikk. 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Last inn data for Oslo og Tromsø ---
df_oslo = pd.read_csv("../../data/processed/vaerdata_oslo.csv")
df_tromso = pd.read_csv("../../data/processed/vaerdata_tromso.csv")

# Legg til by-kolonne
df_oslo["city"] = "Oslo"
df_tromso["city"] = "Tromsø"

# Slå sammen
df = pd.concat([df_oslo, df_tromso], ignore_index=True)

# --- Forbered datasettet ---
df['referenceTime'] = pd.to_datetime(df['referenceTime'])
df['month'] = df['referenceTime'].dt.month
df['day'] = df['referenceTime'].dt.day

# Filtrer relevante målinger
relevante_elementer = [
    'sum(precipitation_amount P1D)',
    'max(air_temperature P1D)',
    'mean(wind_speed P1D)'
]
df = df[df['elementId'].isin(relevante_elementer)]

# --- Interaktive widgets ---
dag_slider = widgets.IntSlider(value=17, min=1, max=31, description='Dag:')
maaned_slider = widgets.IntSlider(value=5, min=1, max=12, description='Måned:')

by_dropdown = widgets.SelectMultiple(
    options=['Oslo', 'Tromsø'],
    value=['Oslo'],
    description='By(er):'
)

knapp = widgets.Button(description="Vis statistikk")
output = widgets.Output()

# --- Funksjon som viser grafen ---
def vis_statistikk(b):
    with output:
        clear_output()
        dag = dag_slider.value
        maaned = maaned_slider.value
        valgte_byer = by_dropdown.value

        dagdata = df[
            (df['month'] == maaned) &
            (df['day'] == dag) &
            (df['city'].isin(valgte_byer))
        ]

        if dagdata.empty:
            print("Ingen data tilgjengelig for valgt dag/by.")
            return

        # Gruppér og finn gjennomsnitt
        stats = dagdata.groupby(['city', 'elementId'])['value'].mean().unstack()

        # Plot
        stats.plot(kind='bar', figsize=(10, 5), edgecolor='black')
        plt.title(f"Gjennomsnittlig vær for {dag:02d}.{maaned:02d}")
        plt.ylabel('Verdi')
        plt.xticks(rotation=0)
        plt.legend(title='Måling')
        plt.grid(axis='y')
        plt.tight_layout()
        plt.show()

        print("\nGjennomsnittsverdier:")
        for by in stats.index:
            print(f"\n{by}:")
            for col in stats.columns:
                print(f"  {col}: {stats.loc[by, col]:.2f}")

# --- Koble knapp til funksjon ---
knapp.on_click(vis_statistikk)

# --- Vis widgets og output ---
display(maaned_slider, dag_slider, by_dropdown, knapp, output)


IntSlider(value=5, description='Måned:', max=12, min=1)

IntSlider(value=17, description='Dag:', max=31, min=1)

SelectMultiple(description='By(er):', index=(0,), options=('Oslo', 'Tromsø'), value=('Oslo',))

Button(description='Vis statistikk', style=ButtonStyle())

Output()